In [70]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import random
import time
import copy
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from tqdm import tqdm
from PIL import Image

In [71]:
def split_dataset(csv_path, train_save_path, valid_save_path, test_size, random_seed):  
    
    # CSV 파일 읽기
    df = pd.read_csv(csv_path, header=None, names=['path', 'label'])

    # 데이터를 train과 val로 나누기
    train_df, valid_df = train_test_split(df, test_size=test_size, random_state=random_seed, stratify=df['label'])

    # 나눠진 데이터를 CSV 파일로 저장
    train_df.to_csv(train_save_path, index=False, header=False)
    valid_df.to_csv(valid_save_path, index=False, header=False)


In [72]:
split_dataset('image-data/labels-map.csv', 'image-data/train-labels.csv', 'image-data/valid-labels.csv', 0.2, 42)

In [73]:
train_transform =   transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

valid_transform =   transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [74]:
class KoreanHandwritingDataset(Dataset):
    
    def __init__(self, csv_file, image_dir, label_file, transform=None):
        self.dataset = pd.read_csv(csv_file, header=None, names=['path', 'label'])
        self.image_dir = image_dir
        self.label_file = label_file
        self.transform = transform

        # 파일에서 한글 글자를 읽어오기
        with open(self.label_file, 'r', encoding='utf-8') as f:
            hangul_chars = [line.strip() for line in f.readlines()]
        
        # 각 한글 글자에 순차적으로 레이블 번호 부여
        self.label_mapping = {char: idx for idx, char in enumerate(hangul_chars)}

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.dataset.iloc[idx]['path'])
        # Gray 이미지를 RGB로 열기
        image = Image.open(img_path).convert("RGB")
        
        if self.transform:
            image = self.transform(image)
                
        label = self.dataset.iloc[idx]['label']
        label = self.label_mapping[label]
        return image, label
    

In [82]:
train_csv_file = "./image-data/train-labels.csv"
valid_csv_file = "./image-data/valid-labels.csv"
label_file = "./labels/256-common-hangul.txt"
image_dir = "./image-data/hangul-images"

train_dataset = KoreanHandwritingDataset(train_csv_file, image_dir, label_file, train_transform)
valid_dataset = KoreanHandwritingDataset(valid_csv_file, image_dir, label_file, valid_transform)

train_size = len(train_dataset)
valid_size = len(valid_dataset)
print(train_size, valid_size)


3276 820


In [83]:

train_data_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
valid_data_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False, num_workers=0)

In [84]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model_ft = models.vgg16(pretrained=True)

cpu


/Users/yeongseonchoe/Documents/GitHub/pytorch-hangul-recognition/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/yeongseonchoe/Documents/GitHub/pytorch-hangul-recognition/venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [96]:
model_ft.classifier[6] = nn.Linear(in_features=4096, out_features=256)

criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [100]:
def train_model(model, data_loader, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode
            running_loss = 0.0
            running_corrects = 0
            # Iterate over data.
            for inputs, labels in tqdm(data_loader[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step()
            epoch_loss = running_loss / len(data_loader[phase].dataset)
            epoch_acc = running_corrects.double() / len(data_loader[phase].dataset)
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            # deep copy the model
        print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [101]:
data_loader = {'train': train_data_loader, 'valid': valid_data_loader}

In [ ]:
model_ft = train_model(model_ft, data_loader, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=25)


Epoch 0/24
----------


100%|██████████████████████████████████████████████████████| 52/52 [16:52<00:00, 19.48s/it]


train Loss: 5.4797 Acc: 0.0125


100%|██████████████████████████████████████████████████████| 13/13 [00:59<00:00,  4.57s/it]


valid Loss: 5.3590 Acc: 0.0646

Epoch 1/24
----------


...

100%|██████████████████████████████████████████████████████| 52/52 [16:47<00:00, 19.37s/it]


train Loss: 0.0414 Acc: 0.9857


100%|██████████████████████████████████████████████████████| 13/13 [01:00<00:00,  4.66s/it]


valid Loss: 0.0084 Acc: 0.9988

Epoch 24/24
----------


 71%|██████████████████████████████████████▍               | 37/52 [12:07<04:57, 19.85s/it]

In [ ]:

def predict_image(model, image_path, transform):
    # 이미지 불러오기
    image = Image.open(image_path).convert("RGB")
    
    # 이미지 전처리
    image = transform(image).unsqueeze(0) # 차원 추가 (배치 차원)
    image = image.to(device)
    
    # 모델 추론 모드 설정 및 예측
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        return predicted.item()

# ResNet34의 입력 사이즈와 일치하도록 이미지 전처리를 위한 transform 정의
transform = transforms.Compose([
    transforms.Resize((224, 224),
    transforms.ToTensor()
])

In [ ]:
image_path = './test-images/test_1.jpeg'
predicted_label = predict_image(model, image_path, transform)
print(f"Predicted Label: {predicted_label}")

In [ ]:
image_path = './test-images/test_2.jpeg'
predicted_label = predict_image(model, image_path, transform)
print(f"Predicted Label: {predicted_label}")

In [ ]:
image_path = './test-images/test_2.jpeg'
predicted_label = predict_image(model, image_path, transform)
print(f"Predicted Label: {predicted_label}")